In [24]:
import pandas as pd
import seaborn as sns

sns.set() 

import logging
import warnings

# Gensim
import gensim
import gensim.corpora as corpora

# Display setting to show more characters in column
pd.set_option('display.max_columns', 500)

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'com', 'http', 'mail', 'pm'])

import pickle
import pyLDAvis
import pyLDAvis.gensim_models

### Load data

In [25]:
data_ready = pickle.load( open( "data/clean_words.pickle", "rb" ) )

### LDA

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]


In [27]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=100,
    alpha="auto",
    iterations=100,
    per_word_topics=True,
)

print(lda_model.print_topics())


In [ ]:
pickle.dump(lda_model, open("data/lda_model_v1.pickle", "wb"))

# Topic


In [ ]:
data = pickle.load( open( "data/data.pickle", "rb" ) )
lda_model = pickle.load( open( "data/lda_model_v1.pickle", "rb" ) )

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    sent_topics_df = pd.DataFrame()

    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]),
                    ignore_index=True,
                )
            else:
                break
    sent_topics_df.columns = ["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"]

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df


In [ ]:
df_topic_sents_keywords = format_topics_sentences(
    ldamodel=lda_model, corpus=corpus, texts=data_ready
)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = [
    "Document_No",
    "Dominant_Topic",
    "Topic_Perc_Contrib",
    "Keywords",
    "Text",
]
df_dominant_topic.head(1)


/var/folders/f_/lq61lr8515sdgzbg0slfc_5c0000gn/T/ipykernel_24820/2689881985.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(
/var/folders/f_/lq61lr8515sdgzbg0slfc_5c0000gn/T/ipykernel_24820/2689881985.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2,0.5456,"day, time, week, give, tell, friend, send, loo...","[sure, happen, impression, visit, enter, trial..."


In [ ]:
sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby("Dominant_Topic")

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat(
        [
            sent_topics_sorteddf_mallet,
            grp.sort_values(["Perc_Contribution"], ascending=False).head(1),
        ],
        axis=0,
    )

# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = [
    "Topic_Num",
    "Topic_Perc_Contrib",
    "Keywords",
    "Representative Text",
]

# Show
sent_topics_sorteddf_mallet.head()


,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.9913,"service, information, employee, time, sit, new...","[start, date, schedule, scheduling, iso, datum..."
1,1,0.9901,"image, font, size, align, img, click, tr, leav...","[document, title_meta, content, text, charset,..."
2,2,0.9903,"day, time, week, give, tell, friend, send, loo...","[struck_chord, date, message, date, time, lkra..."
3,3,0.9981,"power, gas, energy, market, price, company, st...","[datevalue, capacityout, fueltype, alt, coal, ..."
4,4,0.9806,"message, original, send, maggi, forward, recei...","[date, date, date, date, date, date, date, dat..."


### combine df

In [ ]:
email_df = pd.read_csv("data/raw_mail_all.csv")
email_df.head(2)

,id_mail,date,from,to,subject,body,x_origin
0,<17334447.1075857585446.JavaMail.evans@thyme>,"Thu, 16 Nov 2000 09:30:00 -0800 (PST)",msagel@home.com,jarnold@enron.com,Status,John:\n?\nI'm not really sure what happened be...,Arnold-J
1,<19171686.1075857585034.JavaMail.evans@thyme>,"Fri, 8 Dec 2000 05:05:00 -0800 (PST)",slafontaine@globalp.com,john.arnold@enron.com,re:summer inverses,i suck-hope youve made more money in natgas la...,Arnold-J


In [ ]:
email_with_topic = pd.concat([email_df, df_dominant_topic], axis=1)
email_with_topic.to_csv("data/email_with_topic_v1.csv", index=False)

### VIZ

In [ ]:

pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(f"all_v1"))

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(f"all_v1") +'.html')
LDAvis_prepared

/Users/hsinhan/PycharmProjects/Enron_Email_Usecase/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/hsinhan/PycharmProjects/Enron_Email_Usecase/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/hsinhan/PycharmProjects/Enron_Email_Usecase/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/hsinhan/PycharmProjects/Enron_Email_Usecase/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; se

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.128681 -0.187493       1        1  42.422343
0      0.064625 -0.084594       2        1  22.879321
2      0.079974 -0.081054       3        1  18.000223
4      0.158142  0.319607       4        1  13.785962
1     -0.431422  0.033534       5        1   2.912151, topic_info=             Term           Freq          Total Category  logprob  loglift
335       message  777854.000000  777854.000000  Default  30.0000  30.0000
488      original  629839.000000  629839.000000  Default  29.0000  29.0000
91           send  667473.000000  667473.000000  Default  28.0000  28.0000
2755        image  275842.000000  275842.000000  Default  27.0000  27.0000
4156         font  157195.000000  157195.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
1552       season    5318.844267   14552.582428   Topic5  -5.5359   2.5298
3510  unsubscribe    5327.253180   23092.515795   Topic5  -5.5343   2.0696
798     copyright    4324.967704    9817.651257   Topic5  -5.7427   2.7165
175         email    6356.121338  137091.729785   Topic5  -5.3577   0.4650
356         start    5509.214498   54090.033752   Topic5  -5.5007   1.2521

[357 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3573      4  0.999941  acceptance
122       1  0.030536      access
122       2  0.969446      access
307       2  0.755766     address
307       3  0.071525     address
...     ...       ...         ...
8742      3  0.071622        yard
8742      5  0.928171        yard
302       1  0.709490        year
302       2  0.062855        year
302       3  0.227655        year

[603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 5, 2])